## Collaborative/Personalized Learning

In [1]:
%matplotlib inline
import sys
import os
helpers_path = os.path.abspath('../')
if helpers_path not in sys.path:
    sys.path.append(helpers_path)

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import Kernel

from online_kernels.klms import KLMS
from online_kernels.knlms import KNLMS
from online_kernels.krls import KRLS
from online_kernels.klmsRFF import klms_rff
from online_kernels.knlmsRFF import knlms_rff
from online_kernels.krlsRFF import krls_rff

from data.datasets import load_dodd
from sklearn.utils import shuffle

In [92]:
iteration = 1
# Kernel Dictionary size
D = 50
step_size = 0.8
KLMS_RFF = klms_rff(step_size,D)
num_data=300
# N individuals
N = 10
# p features
p = 2
# K canonical models
K = 4
# L 
L = 3

num_experiments = 1
C = np.ones((K,N))
A = np.zeros((D,K))
u,d,d_true = load_dodd(num_data)
kernel = Kernel(3.73)
sigma = 1/np.sqrt(2*kernel.sigma)
W = (1/sigma) * np.random.normal(size=(2,D))
b = np.random.uniform(0,2*np.pi,(D,1))
h = np.sqrt(2/D) * np.cos(W.T @ u.T + b)

for i in tqdm(range(iteration)):
#     v = np.random.randint(len(d))
    B = np.zeros((D,N))
    # Learning A gived fixed Cs (Least Squares)  
    for n in range(N):
        v = np.random.randint(len(d))
        h_n = h[:,v].reshape((D,1))
        d_n = np.array([d[v]])
#         alpha_in = (A@C.T[n]).reshape
        _,alpha_out = KLMS_RFF.train(h_n,d_n,(A@C.T[n]).reshape((D,1)))
        B.T[n] = alpha_out.reshape((D,))
    
    print(A.shape)
    
    A_hat,_,_,_ = np.linalg.lstsq(C.T, B.T,rcond=None) 
    A = A_hat.T
    
    # Learning C given the A are fixed (Difficult)
    # This is an iteration process (nonconvex)
    # Hint: maybe work in the dual space and derive the Lagrangian optimization 
    C = 

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 625.08it/s]

(50, 4)
(4, 50)
(50, 4)


In [14]:
iteration = 1
# N individuals
N = 10
# p features
p = 2
# K canonical models (personalized learning systems)
K = 5
# L 
L = 3
# Kernel Dictionary size
D = 50
num_experiments = 1
C = np.ones((K,N))
A = np.zeros((D,K))
# u,d,d_true = load_dodd(num_data)

# W = (1/sigma) * np.random.normal(size=(2,D))
# b = np.random.uniform(0,2*np.pi,(D,1))
# h = np.sqrt(2/D) * np.cos(W.T @ u.T + b)

for i in tqdm(range(iteration)):
#     v = np.random.randint(len(d))
    
    for edge in range(N):
        # Learning A gived fixed Cs (Least Squares)  
            # Convert c to C
        C_i = np.append(C.T[edge],np.zeros((K*(p-1))))
        for k in range(p-1):
            C_i = np.vstack((C_i,np.roll(np.append(C.T[edge],np.zeros((K*(p-1)))),K*(k+1))))

        # Learn C given the A are fixed (Difficult)
        # This is an iteration process (nonconvex)



        # Calculate a

        # Convert a to A

        # Calculate C
        C = 

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2799.94it/s]


In [48]:
B = np.zeros((D,K))
B.T[0] = np.ones((50,)

(50, 4)

In [59]:
(A@C.T[0]).shape

(50,)

In [68]:
np.linalg.inv(C.T@C)

LinAlgError: Singular matrix